# Setup and EDA Notebook
---

## Import Dependencies

In [ ]:
!pip install mediapipe

Library Requierment

In [1]:
import numpy as np
import json

In [6]:
from matplotlib import pyplot as plt
import pandas as pd
import plotly.express as px

Google Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/data-capstone/asl-signs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Function Helper

* Generate Decoder and Encoder Labels

In [ ]:
def extract_json(json_list):
  encode = {}
  decode = {}
  
  for id, word in enumerate(json_list):
    encode[word.lower()] = id
    decode[id] = word.lower()
  
  return encode, decode

* Adding New Information Data

In [ ]:
def expand_info(data):
  start_frame, end_frame, total_frame = [], [], []
  size = len(data)
  for i, path in enumerate(data['path']):
    print(f'[{i+1}/{size}] {path}')
    df_pq = pd.read_parquet(path)
    start_frame.append(int(df_pq.describe()['frame']['min']))
    end_frame.append(int(df_pq.describe()['frame']['max']))
    total_frame.append(len(df_pq['frame'].value_counts()))
  
  data['start_frame'] = start_frame
  data['end_frame'] = end_frame
  data['total_frame'] = total_frame

  return data

## Setup and Load Data

In [6]:
df_main = pd.read_csv('train.csv')
df_main.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [7]:
with open("sign_to_prediction_index_map.json", 'r') as json_file:
  json_data = json_file.read()

s2p_json = json.loads(json_data)

In [8]:
encode, decode = extract_json(s2p_json)
print(encode)
print(decode)

{'tv': 0, 'after': 1, 'airplane': 2, 'all': 3, 'alligator': 4, 'animal': 5, 'another': 6, 'any': 7, 'apple': 8, 'arm': 9, 'aunt': 10, 'awake': 11, 'backyard': 12, 'bad': 13, 'balloon': 14, 'bath': 15, 'because': 16, 'bed': 17, 'bedroom': 18, 'bee': 19, 'before': 20, 'beside': 21, 'better': 22, 'bird': 23, 'black': 24, 'blow': 25, 'blue': 26, 'boat': 27, 'book': 28, 'boy': 29, 'brother': 30, 'brown': 31, 'bug': 32, 'bye': 33, 'callonphone': 34, 'can': 35, 'car': 36, 'carrot': 37, 'cat': 38, 'cereal': 39, 'chair': 40, 'cheek': 41, 'child': 42, 'chin': 43, 'chocolate': 44, 'clean': 45, 'close': 46, 'closet': 47, 'cloud': 48, 'clown': 49, 'cow': 50, 'cowboy': 51, 'cry': 52, 'cut': 53, 'cute': 54, 'dad': 55, 'dance': 56, 'dirty': 57, 'dog': 58, 'doll': 59, 'donkey': 60, 'down': 61, 'drawer': 62, 'drink': 63, 'drop': 64, 'dry': 65, 'dryer': 66, 'duck': 67, 'ear': 68, 'elephant': 69, 'empty': 70, 'every': 71, 'eye': 72, 'face': 73, 'fall': 74, 'farm': 75, 'fast': 76, 'feet': 77, 'find': 78, '

In [ ]:
# df_expanded = expand_info(df_main)
# df_expanded.to_csv('train_expanded.csv', index=False)

In [7]:
df_expanded = pd.read_csv('train_expanded.csv')
df_expanded.head()

,path,participant_id,sequence_id,sign,start_frame,end_frame,total_frame
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,20,42,23
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,29,39,11
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,103,207,105
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,17,28,12
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,22,39,18


In [11]:
df_pq0 = pd.read_parquet(df_main['path'][0])
df_pq0

,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152
...,...,...,...,...,...,...,...
12484,42,42-right_hand-16,right_hand,16,0.001660,0.549574,-0.145409
12485,42,42-right_hand-17,right_hand,17,0.042694,0.693116,-0.085307
12486,42,42-right_hand-18,right_hand,18,0.006723,0.665044,-0.114017
12487,42,42-right_hand-19,right_hand,19,-0.014755,0.643799,-0.123488


## Exploratory Data Analysis

In [ ]:
df_expanded[['path','sequence_id']].astype(str).describe()

,path,sequence_id
count,94477,94477
unique,94477,94477
top,train_landmark_files/26734/1000035562.parquet,1000035562
freq,1,1


In [ ]:
df_expanded['sign'].describe().to_frame()

,sign
count,94477
unique,250
top,listen
freq,415


In [ ]:
counts = df_expanded['sign'].value_counts().sort_values(ascending=False)
fig = px.histogram(df_expanded, y='sign', color='sign', category_orders={'sign': counts.index})
fig.update_layout(bargap=0.25, height=5000, width=1000, showlegend=False)
fig.update_layout(title={
    'text': "Number of Data Each Sign Language Word",
    'x': 0.5,
    'font': {'size': 20, 'family': 'Arial', 'color': 'black'}
})
fig.update_yaxes(title_text='Sign Words')
fig.show()

In [12]:
df_expanded[['start_frame', 'end_frame', 'total_frame']].describe()

,start_frame,end_frame,total_frame
count,94477.000000,94477.000000,94477.000000
mean,30.421044,67.361379,37.935021
std,29.630199,58.981009,44.177069
min,0.000000,1.000000,2.000000
25%,16.000000,33.000000,12.000000
50%,21.000000,43.000000,22.000000
75%,29.000000,62.000000,44.000000
max,484.000000,546.000000,537.000000


In [13]:
mean_frames = pd.DataFrame(df_expanded.groupby('sign')['total_frame'].mean().sort_values())

fig = px.bar(y=mean_frames.index, x=df_expanded.groupby('sign')['total_frame'].mean().sort_values(), color=mean_frames.index)
fig.update_layout(bargap=0.25, height=5000, width=1000, showlegend=False)
fig.update_layout(xaxis_title='Average Frame', yaxis_title='Sign')
fig.update_layout(title={
    'text': "Average Frame Each Sign Language Word",
    'x': 0.5,
    'font': {'size': 20, 'family': 'Arial', 'color': 'black'}
})
fig.show()